In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

actions = np.array(['BOIRE', 'JE_TAIME', 'MERCI','JOUER','AMOUR'])
label_map = {label:num for num, label in enumerate(actions)}


print(label_map)

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# Actions that we try to detect
actions = np.array(['BOIRE', 'JE_TAIME', 'MERCI','JOUER','AMOUR'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

print(np.array(sequences).shape)

print(np.array(labels).shape)

X = np.array(sequences)

y = to_categorical(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))



res = [.7, 0.2, 0.1]

actions[np.argmax(res)]
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
model.summary()

model.save('action1.h5')

model.load_weights('action1.h5')


from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
print(multilabel_confusion_matrix(ytrue, yhat))
print(accuracy_score(ytrue, yhat))


colors = [(245,117,16), (117,245,16), (16,100,245),(245,117,16), (117,245,16)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return output_frame

plt.figure(figsize=(18,18))

{'BOIRE': 0, 'JE_TAIME': 1, 'MERCI': 2, 'JOUER': 3, 'AMOUR': 4}
(150, 30, 1662)
(150,)
Epoch 1/2000
5/5 [==============================] - 4s 111ms/step - loss: 1.8170 - categorical_accuracy: 0.1831
Epoch 2/2000
5/5 [==============================] - 1s 104ms/step - loss: 2.1947 - categorical_accuracy: 0.1197
Epoch 3/2000
5/5 [==============================] - 1s 101ms/step - loss: 3.3094 - categorical_accuracy: 0.2324
Epoch 4/2000
5/5 [==============================] - 1s 102ms/step - loss: 1.6058 - categorical_accuracy: 0.2183
Epoch 5/2000
5/5 [==============================] - 0s 103ms/step - loss: 1.5973 - categorical_accuracy: 0.2324
Epoch 6/2000
5/5 [==============================] - 1s 103ms/step - loss: 1.6428 - categorical_accuracy: 0.1761
Epoch 7/2000
5/5 [==============================] - 1s 104ms/step - loss: 1.5866 - categorical_accuracy: 0.3169
Epoch 8/2000
5/5 [==============================] - 1s 117ms/step - loss: 1.5647 - categorical_accuracy: 0.2606
Epoch 9/2000
5/5 

<Figure size 1296x1296 with 0 Axes>

<Figure size 1296x1296 with 0 Axes>